<a href="https://colab.research.google.com/github/RMeuter/NLPHumourProjet/blob/ironie/ironie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTS AND GLOBAL VARIABLES

In [1]:
!pip3 install langdetect
!pip3 install stanza

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import stanza
from langdetect import *

FILENAME = "/content/drive/MyDrive/COLAB_STUFF/SMS/88milSMS_88522.csv"

In [2]:
stanza.download('fr') 
nlp = stanza.Pipeline('fr')

2021-02-24 15:56:01 INFO: Downloading default packages for language: fr (French)...
2021-02-24 15:56:02 INFO: File exists: /root/stanza_resources/fr/default.zip.
2021-02-24 15:56:09 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-02-24 15:56:09 INFO: Loading these models for language: fr (French):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |
| ner       | wikiner |

2021-02-24 15:56:09 INFO: Use device: cpu
2021-02-24 15:56:09 INFO: Loading: tokenize
2021-02-24 15:56:09 INFO: Loading: mwt
2021-02-24 15:56:09 INFO: Loading: pos
2021-02-24 15:56:09 INFO: Loading: lemma
2021-02-24 15:56:09 INFO: Loading: depparse
2021-02-24 15:56:10 INFO: Loading: ner
2021-02-24 15:56:12 INFO: Done loading processors!


# LOADING DATASET

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
df = pd.read_csv(FILENAME)
del df['Unnamed: 3']
df = df.rename(columns ={"SMS_ANON": "sms"})

In [40]:
df.head()

,NUM_SMS,DATETIME,ID_NUM_TEL,sms
0,1,15 sept. 2011 07:28:55,477,Hey ca va?\nAlors cette rentree?\nVa falloir s...
1,2,15 sept. 2011 08:02:08,477,Ok super merci! Oui j'y comprends rien du tout...
2,3,15 sept. 2011 08:03:01,477,Coucou !\nC'est quand la feria de Nimes? \nJ's...
3,4,15 sept. 2011 08:03:35,477,Coucou :)\nOui ca c'est bien passe!\nAlors je ...
4,5,15 sept. 2011 09:05:28,477,On peut se rejoindre quelque part? Tu as cours...


Certains sms sont écrit en anglais, il faut donc les supprimer du corpus pour pouvoir traiter les données.

In [41]:
detect(df.SMS[0])

AttributeError: ignored

In [42]:
def detect_lang(text):
    try:
        return detect(text)
    except:
        return None

df['langue'] = df["sms"].apply(lambda x : detect_lang(x))
df.head()

,NUM_SMS,DATETIME,ID_NUM_TEL,sms,langue
0,1,15 sept. 2011 07:28:55,477,Hey ca va?\nAlors cette rentree?\nVa falloir s...,fr
1,2,15 sept. 2011 08:02:08,477,Ok super merci! Oui j'y comprends rien du tout...,fr
2,3,15 sept. 2011 08:03:01,477,Coucou !\nC'est quand la feria de Nimes? \nJ's...,fr
3,4,15 sept. 2011 08:03:35,477,Coucou :)\nOui ca c'est bien passe!\nAlors je ...,fr
4,5,15 sept. 2011 09:05:28,477,On peut se rejoindre quelque part? Tu as cours...,fr


In [43]:
df.isnull().sum()

NUM_SMS         0
DATETIME        0
ID_NUM_TEL      0
sms             1
langue        410
dtype: int64

In [44]:
df.loc[df['langue'] != 'fr']

,NUM_SMS,DATETIME,ID_NUM_TEL,sms,langue
27,28,15 sept. 2011 10:56:48,88,La j'ai fini je regarde le film,sl
32,33,15 sept. 2011 10:59:17,136,<TEL_10> texto <PRE_6> vite vite pour le casti...,en
38,39,15 sept. 2011 11:01:29,353,Coucou! ça va? Balade vers 5h?,ca
44,45,15 sept. 2011 11:03:40,136,"Ok ca marche, il va avoir quel age ? :)",it
50,51,15 sept. 2011 12:04:16,254,Je t'aiiiime...,et
...,...,...,...,...,...
88503,93064,15 déc. 2011 23:11:00,508,Moi je dors lol,sl
88506,93067,15 déc. 2011 23:14:13,508,Merci good night ^^,cy
88512,93074,15 déc. 2011 23:20:44,194,Reveil!!!!,no
88515,93078,15 déc. 2011 23:22:40,194,О ватаяъ sa gere dercire en russe:-P,no


On remarque que beaucoups de texte fr sont annotés en tant que langue différente.
Dans le cadre d'améliorer les résultats des qu'un bout de texte est détecté en Fr alors le texte est annoté FR 

In [45]:
detect_langs(df.loc[23,'sms'])

[fr:0.7142830902760495, it:0.1428581608129409, ca:0.14285754376453824]

In [46]:
for n in range(len(detect_langs(df.loc[23,'sms']))):
    print(detect_langs(df.loc[23,'sms'])[n].lang)
#detect_langs(df.loc[23,'sms'])[0].lang

fr
it
hr
hr


In [47]:
def liste_langue(langs):
    l = []
    for n in catch_detectlangs(langs):
        l.append(n.lang)
    return l

In [48]:
liste_langue(df.loc[23,'sms'])

['fr', 'lt', 'ca', 'hr', 'it']

In [49]:
'fr' in liste_langue(df.loc[23,'sms'])

True

In [50]:
detect_langs(df.loc[99,'sms'])

LangDetectException: ignored

In [51]:
def catch_detectlangs(texte):
    try:
        l = detect_langs(texte)
    except:
        l = []
        
    return l

In [52]:
catch_detectlangs(df.loc[100,'sms'])

[fr:0.8571393516191088, en:0.14285591306338885]

In [53]:
'fr' in liste_langue(df.loc[1,'sms'])

True

In [ ]:
for i in range(len(df)):
    if('fr' in liste_langue(df.loc[i,'sms'])):
       df.loc[i,'langue'] = 'fr'
    else:
        try:
            df.loc[i,'langue'] = detect(df.loc[i,'sms'])
        except:
            df.loc[i,'langue'] = None

In [ ]:
df.loc[df['langue'] != 'fr'].head()

In [ ]:
df.loc[125,:]